In [1]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,LSTM
from tensorflow.keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import numpy as np
import sys

Using TensorFlow backend.


In [2]:
file_names=['AIIMAT.txt', 'MLOE.txt','OKEWFSMP.txt','TAM.txt','TAMatter.txt', 'THWP.txt','TPP.txt']
corpus=[]
def get_corpus(path):
    with open(path,'r',errors='ignore') as data:
        corpus=data.read()
        corpus=corpus.lower()
    return corpus


In [3]:
for i in range(len(file_names)):
    path='/Users/phuongqn/Desktop/INF552/Homework/Homework 7 Data/Book Files/books/'+file_names[i]+''
    corpus.append(get_corpus(path))

In [4]:
print(corpus[4])


the analysis of matter 


the nature of the problem 
apart from pure mathematics, the most advanced of the 
sciences is physics. certain parts of theoretical physics have 
reached the point which makes it possible to exhibit a logical 
chain from certain assumed premisses to consequences appar- 
ently very remote, by means of purely mathematical deduc- 
tions. this is true especially of everything that belongs to 
the general theory of relativity. it cannot be said that physics 
as a whole has yet reached this stage, since quantum pheno- 
mena, and the existence of electrons and protons, remain, for 
the moment, brute facts. but perhaps this state of affairs 
will not last long; it is not chimerical to hope that a unified 
treatment of the whole of physics may be possible before many 
years have passed. 

in spite, however, of the extraordinary successes of physics 
considered as a science, the philosophical outcome is much less 
dear than it seemed to be when less was known. the purp

In [23]:
def clean_text(text):
    text = text.translate(string.punctuation)
    
    text = re.sub(r"[^A-Za-z0-9^!\/'+-=:;<]", " ", text)
    text = re.sub(r"what's", "what is ", text)
    text = re.sub(r"\'s", " ", text)
    text = re.sub(r"\'ve", " have ", text)
    text = re.sub(r"n't", " not ", text)
    text = re.sub(r"i'm", "i am ", text)
    text = re.sub(r"\'re", " are ", text)
    text = re.sub(r"\'d", " would ", text)
    text = re.sub(r"\'ll", " will ", text)
    text = re.sub(r",", " ", text)
    text = re.sub(r"\.", " ", text)
    text = re.sub(r"!", " ", text)
    text = re.sub(r"\/", " ", text)
    text = re.sub(r"\^", " ", text)
    text = re.sub(r"\+", " ", text)
    text = re.sub(r"\-", " ", text)
    text = re.sub(r"\=", " ", text)
    text = re.sub(r"<", " ", text)
    text = re.sub(r";", " ", text)
    text = re.sub(r"'", " ", text)
#     text = re.sub(r"(\d+)(k)", r"\g<1>000", text)
    text = re.sub(r":", " ", text)
#     text = re.sub(r"\0s", "0", text)
#     text = re.sub(r"\s{2,}", " ", text)
    text = re.sub(r"[0123456789]", " ", text)
    
    return text

In [24]:
import string
import re
c0 = clean_text(corpus[0])
c1 = clean_text(corpus[1])
c2 = clean_text(corpus[2])
c3 = clean_text(corpus[3])
c4 = clean_text(corpus[4])
c5 = clean_text(corpus[5])
c6 = clean_text(corpus[6])
corpus_c=[c0,c1,c2,c3,c4,c5,c6]

In [25]:
# chars = sorted(list(set(corpus[0])))
chars=[]
for i in corpus_c:
    chars.extend(list(set(set(i))))
    
chars = sorted(list(set(chars)))
char_int = dict((c, i) for i, c in enumerate(chars))
int_char = dict((i, c) for i, c in enumerate(chars))

In [26]:
char_int

{' ': 0,
 'a': 1,
 'b': 2,
 'c': 3,
 'd': 4,
 'e': 5,
 'f': 6,
 'g': 7,
 'h': 8,
 'i': 9,
 'j': 10,
 'k': 11,
 'l': 12,
 'm': 13,
 'n': 14,
 'o': 15,
 'p': 16,
 'q': 17,
 'r': 18,
 's': 19,
 't': 20,
 'u': 21,
 'v': 22,
 'w': 23,
 'x': 24,
 'y': 25,
 'z': 26}

In [27]:
def i_o(file, size):
    ip=[]
    op=[]
    for i in range(0, len(file) - size+1, 1):
        seq_in = file[i:i + size - 1]
        seq_out = file[i + size-1]
        ip.append([char_int[c] for c in seq_in])
        op.append(char_int[seq_out])
    return ip, op

In [ ]:
ip_seq=[]
op_char=[]
for c in corpus_c:
    w=100
    ip, op = i_o(c, w)
    ip_seq.extend(ip)
    op_char.extend(op)

In [ ]:
# reshape X to be [samples, time steps, features]
X = np.reshape(ip_seq, (len(ip_seq), 99,1))
# normalize
X = X / float(len(chars))
# one hot encode the output variable
y = np_utils.to_categorical(op_char)
print(X.shape)

In [ ]:
LSTMmodel = Sequential()
LSTMmodel.add(LSTM(124, input_shape=(X.shape[1], X.shape[2])))
# LSTMmodel.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), dropout=0.2))
LSTMmodel.add(Dropout(0.2))
LSTMmodel.add(Dense(y.shape[1], activation='softmax'))
print(LSTMmodel.summary())
LSTMmodel.compile(loss='categorical_crossentropy', optimizer='adam')



In [11]:
filepath= '/Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-{epoch:02d}-{loss:.4f}.hdf5'
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

This below cell takes super long to run, about ranging from 5-8hrs per epoch. I can't really pinpoint the problem as to why it takes so long, so I stopped it at epoch 8 and use the lowest weight to run the model. Currently doing the same on a prepocessed corpus.

In [ ]:
LSTMmodel.fit(X, y, epochs=30, batch_size=64, callbacks=callbacks_list)

In [15]:
file='/Users/phuongqn/Desktop/INF552/HW-Phuong/LSTMweights/weights-improvement-07-1.8438.hdf5'
LSTMmodel.load_weights(file)
LSTMmodel.compile(loss='categorical_crossentropy', optimizer='adam')

In [1]:
init = 'There are those who take mental phenomena naively, just as they would physical phenomena. This school of psychologists tends not to emphasize the object.'

pattern = [char_int[c] for c in init[-99:].lower()]

for i in range(1000):
    seq = np.reshape(pattern, (1, len(pattern), 1))
    seq = seq / float(len(chars))
    charpredict = LSTMmodel.predict(seq, verbose=0)
    ind = np.argmax(charpredict)
    init+=int_char[ind]
    pattern.append(ind)
    pattern = pattern[1:len(pattern)]

print(init)

NameError: name 'char_int' is not defined